In [1]:
from transformers import AutoTokenizer
import os
import json
import pickle
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer

from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from preprocess import preprocess_retrieval, tokenizer_filter

In [2]:
model_name = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
with open("../data/wikipedia_documents.json", "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))

In [4]:
contexts[0]

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [5]:
from collections import Counter

In [6]:
txt = "나는 뷀뷁 뙇 뛓 이다."
encoded = tokenizer(txt)['input_ids'][1:-1]
tokenized = tokenizer.tokenize(txt)
print(tokenizer.decode(encoded))
print(encoded)
print(tokenized)
print(len(encoded), len(tokenized))

나는 <unk> <unk> <unk> 이다.
[37231, 6, 3, 6, 3, 6, 3, 6, 5769, 5]
['▁나는', '▁', '뷀뷁', '▁', '뙇', '▁', '뛓', '▁', '이다', '.']
10 10


띄어쓰기(_) : 6, UNK : 3 & UNK에 대해서 tokenize는 되지만 encoding은 안되는 듯

In [7]:
preprocessed = []
tokens = []

for context in tqdm(contexts):
    preprocessed.append(preprocess_retrieval(context))
    
for context in tqdm(preprocessed):
    tokens.extend(tokenizer.tokenize(context))
    
counter = Counter(tokens)
len(counter)

Token indices sequence length is longer than the specified maximum sequence length for this model (1090 > 512). Running this sequence through the model will result in indexing errors


58894

In [8]:
type(counter)

collections.Counter

In [9]:
counter.most_common(50)

[('▁', 1135789),
 ('의', 393949),
 ('을', 282722),
 ('에', 269607),
 ('이', 212962),
 ('은', 195973),
 ('는', 182049),
 ('년', 176928),
 ('를', 175570),
 ('로', 154598),
 ('가', 150453),
 ('에서', 138644),
 ('▁이', 118016),
 ('으로', 115491),
 ('한', 107940),
 ('고', 99520),
 ('과', 97610),
 ('인', 97240),
 ('도', 89708),
 ('와', 86126),
 ('월', 83608),
 ('리', 78023),
 ('지', 76937),
 ('일', 75681),
 ('사', 74154),
 ('스', 74053),
 ('▁수', 72838),
 ('기', 65315),
 ('다', 64811),
 ('▁있다', 64410),
 ('어', 60382),
 ('했다', 57025),
 ('시', 56950),
 ('르', 55938),
 ('하였다', 55020),
 ('▁그', 54385),
 ('자', 54093),
 ('하는', 53734),
 ('라', 52647),
 ('해', 51633),
 ('▁전', 51362),
 ('하고', 50673),
 ('이다', 47929),
 ('부', 47854),
 ('하여', 46230),
 ('군', 45556),
 ('▁1', 45034),
 ('▁가', 44516),
 ('▁사', 44447),
 ('대', 44442)]

In [10]:
len(tokens)

22725190

In [11]:
import re

def preprocess_retrieval(corpus):
    corpus = corpus.replace(f"\n", "")
    corpus = re.sub(f"[\"<>\[\].,?!\(\)\:#\|'\=-]", " ", corpus)
    corpus = ' '.join(corpus.split())
    return corpus

In [12]:
sent = preprocess_retrieval(contexts[41132])
print(tokenizer.tokenize(sent))

['▁동', '부', '검', '은', '손', '타', '마', '린', '은', '▁18', '07', '년', '▁다른', '▁타', '마', '린', '과', '▁함께', '▁독일', '▁과학', '자', '▁요', '한', '▁호', '프', '만', '제', '크', '▁Johann', '▁Cent', 'uri', 'us', '▁Hoffman', 'nse', 'gg', '▁가', '▁타', '마', '린', '속', '의', '▁모', '식', '종', '으로', '▁기술', '했다', '▁나중에', '▁동', '부', '검', '은', '손', '타', '마', '린', '은', '▁검', '은', '타', '마', '린', '▁Sa', 'guin', 'us', '▁ni', 'ger', '▁의', '▁이', '명', '으로', '▁간', '주', '되었다', '▁2013', '년', '▁동', '부', '검', '은', '손', '타', '마', '린', '은', '▁분', '자', '생', '물', '학', '▁정보', '와', '▁', '털', '▁', '색', '의', '▁차이', '때', '문', '에', '▁마', '침', '내', '▁재', '검', '증', '되었다', '▁동', '부', '검', '은', '손', '타', '마', '린', '은', '▁3', '개의', '▁기', '아', '나', '▁기', '아', '나', '와', '▁아마', '존', '강', '▁하', '류', '▁사이', '의', '▁지역', '에서', '▁발견', '되는', '▁근', '연', '종', '▁', '붉', '은', '손', '타', '마', '린', '▁Sa', 'guin', 'us', '▁mida', 's', '▁', '싱', '구', '강', '과', '▁토', '칸', '칭', '스', '강', '▁사이에', '▁분', '포', '하는', '▁검', '은', '타', '마', '린', '과', '▁함께', '▁타', '마', '린', '속

In [13]:
common_list = [txt for txt, _ in counter.most_common(50)]
len(common_list)

50

In [14]:
print(common_list)

['▁', '의', '을', '에', '이', '은', '는', '년', '를', '로', '가', '에서', '▁이', '으로', '한', '고', '과', '인', '도', '와', '월', '리', '지', '일', '사', '스', '▁수', '기', '다', '▁있다', '어', '했다', '시', '르', '하였다', '▁그', '자', '하는', '라', '해', '▁전', '하고', '이다', '부', '하여', '군', '▁1', '▁가', '▁사', '대']


In [15]:
commons = common_list
def tokenizer_func(corpus):
    tokenized = tokenizer.tokenize(corpus)
    filtered = [token for token in tokenized if token not in commons]
    return filtered

In [16]:
print(tokenizer_func(preprocessed[41132]))

['▁동', '검', '손', '타', '마', '린', '▁18', '07', '▁다른', '▁타', '마', '린', '▁함께', '▁독일', '▁과학', '▁요', '▁호', '프', '만', '제', '크', '▁Johann', '▁Cent', 'uri', 'us', '▁Hoffman', 'nse', 'gg', '▁타', '마', '린', '속', '▁모', '식', '종', '▁기술', '▁나중에', '▁동', '검', '손', '타', '마', '린', '▁검', '타', '마', '린', '▁Sa', 'guin', 'us', '▁ni', 'ger', '▁의', '명', '▁간', '주', '되었다', '▁2013', '▁동', '검', '손', '타', '마', '린', '▁분', '생', '물', '학', '▁정보', '털', '색', '▁차이', '때', '문', '▁마', '침', '내', '▁재', '검', '증', '되었다', '▁동', '검', '손', '타', '마', '린', '▁3', '개의', '▁기', '아', '나', '▁기', '아', '나', '▁아마', '존', '강', '▁하', '류', '▁사이', '▁지역', '▁발견', '되는', '▁근', '연', '종', '붉', '손', '타', '마', '린', '▁Sa', 'guin', 'us', '▁mida', 's', '싱', '구', '강', '▁토', '칸', '칭', '강', '▁사이에', '▁분', '포', '▁검', '타', '마', '린', '▁함께', '▁타', '마', '린', '속', '붉', '손', '타', '마', '린', '▁S', '▁mida', 's', '▁군', '▁포함', '된다', '▁동', '검', '손', '타', '마', '린', '▁검', '타', '마', '린', '▁종', '분', '화', '▁과정에서', '▁발생', '했고', '▁토', '칸', '칭', '강', '▁두', '▁종', '▁지', '적으로', '▁서로', '▁

In [17]:
def explore_preprocess(corpus):
    lst = ["Original", "Preprocessed", "Filtered"]
    original = tokenizer.tokenize(corpus)
    preprocessed = tokenizer.tokenize(preprocess_retrieval(corpus))
    filtered = tokenizer_func(preprocess_retrieval(corpus))
    sents = [original, preprocessed, filtered]
    for i in range(3):
        print('-'*50, lst[i] , '-'*50)
        print(sents[i])
        print(f"Token Length : {len(sents[i])}")

In [18]:
explore_preprocess(contexts[0])

-------------------------------------------------- Original --------------------------------------------------
['▁이', '▁문서', '는', '▁나라', '▁목록', '이며', ',', '▁전', '▁세계', '▁206', '개', '▁나라', '의', '▁각', '▁현', '황', '과', '▁주', '권', '▁승인', '▁정보를', '▁개', '요', '▁형태', '로', '▁나', '열', '하고', '▁있다', '.', '▁이', '▁목록', '은', '▁명', '료', '화를', '▁위해', '▁두', '▁부분', '으로', '▁나', '뉘', '어', '▁있다', '.', '▁#', '▁첫', '▁번째', '▁부분', '은', '▁바', '티', '칸', '▁시', '국', '과', '▁팔', '레', '스타', '인', '을', '▁포함', '하여', '▁유', '엔', '▁등', '▁국제', '▁', '기구', '에', '▁가입', '되어', '▁국제', '적인', '▁승인', '을', '▁', '널', '리', '▁받았다', '고', '▁여기', '는', '▁195', '개', '▁나라', '를', '▁나', '열', '하고', '▁있다', '.', '▁#', '▁두', '▁번째', '▁부분', '은', '▁일부', '▁지역', '의', '▁주', '권을', '▁사실', '상', '▁(', '데', '▁', '팍', '토', ')', '▁행사', '하고', '▁있지만', ',', '▁아직', '▁국제', '적인', '▁승인', '을', '▁', '널', '리', '▁', '받', '지', '▁않았다', '고', '▁여기', '는', '▁11', '개', '▁나라', '를', '▁나', '열', '하고', '▁있다', '.', '▁두', '▁목록', '은', '▁모두', '▁가', '나', '다', '▁순', '이다', '.', '▁일부', '▁국가', 

In [19]:
import pickle
# from preprocess import tokenizer_filter

with open("../data/tfidv.bin", "rb") as file:
    tfidfv = pickle.load(file)
with open("../data/sparse_embedding.bin", "rb") as file:
    p_embedding = pickle.load(file)

In [21]:
import numpy as np

def get_relevant_doc(query, k=1):
    query_vec = tfidfv.transform([query])
    result = query_vec * p_embedding.T
    result = result.toarray()

    sorted_result = np.argsort(result.squeeze())[::-1]
    doc_score = result.squeeze()[sorted_result].tolist()[:k]
    doc_indices = sorted_result.tolist()[:k]
    return doc_score, doc_indices

In [28]:
def retrieval(query_or_dataset, topk):
    doc_scores, doc_indices = get_relevant_doc(query_or_dataset, k=topk)
    print("[Search query]\n", query_or_dataset, "\n")

    for i in range(topk):
        print(f"Top-{i+1} passage with score {doc_scores[i]:4f}")
        print(contexts[doc_indices[i]])

    # return (doc_scores, [contexts[doc_indices[i]] for i in range(topk)])

In [32]:
print(retrieval("트랜스포머", 3))

[Search query]
 트랜스포머 

Top-1 passage with score 0.560885
트랜스포머 실시영화 세계관에서는 1에서 처음등장하였다. 비클모드는 머스탱이다. 실사판 영화에서는 몸속의 프렌지를 데리고 다닌다 (언뜻 보면 사운드웨이브처럼 숨기고 다닌다)트랜스포머 실사판 3편에서도 잠깐 휠잭를 죽일때나온다. 그리고 군대쪽에서 총을 사격하자 그의 눈에 맞는다. 그리고 군대쪽에서 빌딩에서 내려오고 군대쪽에서 바리케이드의 발쪽에 이상한 폭파무기를 다리쪽에 심어놓고는 폭파되어 사망한다. 그후로 트랜스포머5(최후의 기사)에서도 부활하여 다시 출연한다. 하지만 초반에는 메가트론과 좀 나오다가 중간에 시가전을 펼칠때 그림록의 꼬리에 밀려나간 후로부터 나오질 않는다. 그 다음에는 스톤헨지에서 잠깐 메가트론,니트로제우스와 함께 포착되고 그 후로부터 등장이 없다.

얼라인드 세계관에서 작품인 트랜스포머: 워 포 사이버트론에서는 플레이캐릭터로 등장한다.
Top-2 passage with score 0.499154
1969년, 그렇게 개발된 것이 2254E 컴프레서/리미터이다. 2254는 기본적으로 시그널이 게인의 변화를 주는 섹션을 지나서 다시 게인컨트롤 섹션에 영향을 주는 피드백 방식이었다. 2254의 특징은 총 4개의 트랜스포머로 가득한 게인스테이지 덩어리라는 것이다. 일단 인풋 게인 스테이지에서 첫 번째 트랜스포머가 들어온 신호에 색을 칠한다. 그 다음 신호는 게인 리덕션 유닛으로 넘어오는데 니브만의 다이오드 브릿지(diode bridges)방식은 사실 레벨 효율이 매우 떨어지는 방식이었다. 게인 리덕션 유닛을 지난 신호는 인풋신호에 비해 거의 40dB 가까이 떨어진채 두 번째 트랜스포머의 색을 묻힌 후 니브의 전매특허 BA283 아웃풋 앰프로 들어가게 된다. BA283에 포함된 총 세 번째 트랜스포머를 지나 원래의 라인신호 레벨을 되찾은 신호는 엔지니어 마음에 따라 리미터 사이드체인으로 들어가거나 말거나 중에 하나를 택한다. 리미터까지 들어가면 총 4개